In [ ]:

from PIL import Image
import matplotlib.pyplot as plt
import torch
import json
import requests
import os
import torch
from torchvision import transforms
from transformers import OFATokenizer, OFAModel

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [ ]:
ckpt_dir = "/Users/junwu/Documents/ECS270/Project/Image-Model-with-Grounding-DINO/OFA-tiny"  # path to local OFA-base directory
tokenizer = OFATokenizer.from_pretrained(ckpt_dir)
model = OFAModel.from_pretrained(ckpt_dir) # .to(device)
model.eval()

In [ ]:
coco_dataset = open("../instances_val2017.json")
coco_json = json.load(coco_dataset)
coco_images = coco_json["images"]
coco_annotations = coco_json["annotations"]
coco_categories = coco_json["categories"]
inputs_url = []
image_url = []

category_validation_images = {}
id_category_mapping = {}

# Get mappings of id number and category name
for category in coco_categories:
    category_validation_images[category["id"]] = []
    id_category_mapping[category["id"]] = category["name"]

# For each category, get all images with that category annotation
for images in coco_images:
    image_inputs = []
    for anno in coco_annotations:
        if anno["image_id"] == images["id"]:
            for categories in coco_categories:
                if anno["category_id"] == categories["id"]:
                    if images not in category_validation_images[anno["category_id"]]:
                        category_validation_images[anno["category_id"]].append(images)
                        

In [ ]:
num_cropped_images = 2

open("OFA_cropped_output.txt", "w").close() # clear the text file

# Number of validation images in the category to go through
num_validation_images = 1
category_ids = [20, 40, 60, 80]
# Go through every category
for id in category_ids:
    # All validation images under the category
    images = category_validation_images[id]
    category = id_category_mapping[id]

    # Go thruogh cropped images
    for i in range(num_cropped_images):
        # Open cropped image
        cropped_file = f"../input-data/cropped_objects/{category}_{i + 1}.jpg"
        prompt_image = Image.open(cropped_file)
        patch_img = transform(prompt_image).unsqueeze(0)  # shape: (1, 3, H, W)

        # Prepare text prompt 
        question = " what does the image describe?"
        inputs = tokenizer(question, return_tensors="pt").input_ids

        # Generate caption 
        outputs = model.generate(
            input_ids=inputs,
            patch_images=patch_img,
            patch_masks=torch.tensor([True]),
            num_beams=5,
            max_length=50,
            no_repeat_ngram_size=3,
        )

        # Decode and print result 
        caption = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # Go through validation iamges
        for i in range(num_validation_images):
            with open("OFA_output.txt", "a") as f:
                f.write(f"{images[i]["coco_url"]} || {caption} || {category} || {cropped_file}\n")